In [2]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets.qm9 import QM9
import torch_geometric.datasets.qm9 as qm9
from torch_geometric.data import DataLoader
import torch_geometric.nn as tgnn
from torch_scatter import scatter
import tqdm
import numpy as np
import wandb

In [3]:
from torch_geometric.nn.models.schnet import GaussianSmearing
from dfs_transformer import EarlyStopping

In [4]:
# [0] Reports MAE in eV / Chemical Accuracy of the target variable U0. 
# The chemical accuracy of U0 is 0.043 see [1, Table 5].

# Reproduced table [0]
# MXMNet: 0.00590/0.043 = 0.13720930232558143
# HMGNN:  0.00592/0.043 = 0.13767441860465118
# MPNN:   0.01935/0.043 = 0.45
# KRR:    0.0251 /0.043 = 0.5837209302325582
# [0] https://paperswithcode.com/sota/formation-energy-on-qm9
# [1] Neural Message Passing for Quantum Chemistry, https://arxiv.org/pdf/1704.01212v2.pdf
# MXMNet https://arxiv.org/pdf/2011.07457v1.pdf
# HMGNN https://arxiv.org/pdf/2009.12710v1.pdf
# MPNN https://arxiv.org/pdf/1704.01212v2.pdf
# KRR HDAD kernel ridge regression https://arxiv.org/pdf/1702.05532.pdf
# HDAD means HDAD (Histogram of distances, anglesand dihedral angles)

# [2] Reports the average value of MAE / Chemical Accuracy of over all targets
# [2] https://paperswithcode.com/sota/drug-discovery-on-qm9
target_dict = {0: 'mu, D, Dipole moment', 
               1: 'alpha, {a_0}^3, Isotropic polarizability', 
               2: 'epsilon_{HOMO}, eV, Highest occupied molecular orbital energy',
               3: 'epsilon_{LUMO}, eV, Lowest unoccupied molecular orbital energy',
               4: 'Delta, eV, Gap between HOMO and LUMO',
               5: '< R^2 >, {a_0}^2, Electronic spatial extent',
               6: 'ZPVE, eV, Zero point vibrational energy', 
               7: 'U_0, eV, Internal energy at 0K',
               8: 'U, eV, Internal energy at 298.15K', 
               9: 'H, eV, Enthalpy at 298.15K',
               10: 'G, eV, Free energy at 298.15K',  
               11: 'c_{v}, cal\(mol K), Heat capacity at 298.15K'}

chemical_accuracy = {idx:0.043 for idx in range(12)}
chemical_accuracy[0] = 0.1
chemical_accuracy[1] = 0.1
chemical_accuracy[5] = 1.2
chemical_accuracy[6] = 0.0012
chemical_accuracy[11] = 0.050

In [6]:
wandb.init(project='QM9-GAT', entity='chrisxx')
config = wandb.config
config.hidden_dim = 128
config.nlayers = 3
config.nhead = 1
config.lr = 0.0003
config.n_epochs = 5000
config.patience = 5
config.factor = 0.95
config.minimal_lr = 6e-8
config.target_idx = 7
config.batch_size = 256
config.valid_patience = 200
config.valid_minimal_improvement=0.005
config.n_train = 110000
config.n_valid = 10000
config.model_dir = '../models/qm9/MPNN/noDFS/2/'
config.num_workers = 4
config.dfs_codes = None
config.use_pos = False
config.use_dist = True
config.comment = "Actually, in this version I modified how the readout function is used. Instead of computing "\
"the target property as the weighted sum of target properties, I compute a hidden representation first and the "\
"prediction subsequently."

wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [8]:
dfs_codes = None
if config.dfs_codes is not None:
    import json
    with open(config.dfs_codes, 'r') as f:
        dfs_codes = json.load(f)

In [9]:
def transform(data, dfs_codes = dfs_codes, edge_transform = GaussianSmearing(0, 10, 50), use_dist=config.use_dist):
    features = data.x
    # make atomic number a one hot
    atomic_number = nn.functional.one_hot(features[:, 5].long(), 100)
    # make num_h a one hot
    num_h = nn.functional.one_hot(features[:, -1].long(), 9)
    data.x = torch.cat((features[:, :5], features[:, 6:-1], atomic_number, num_h), axis=1)
    if dfs_codes is not None:
        dfs_indices = nn.functional.one_hot(torch.LongTensor(dfs_codes[data.name]['dfs_indices']), 29)
        data.x = torch.cat((data.x, dfs_indices), axis=1)
    if use_dist:
        row, col = data.edge_index
        edge_weights = (data.pos[row] - data.pos[col]).norm(dim=-1)
        dist_feats = edge_transform(edge_weights)
        data.edge_attr = torch.cat((data.edge_attr, dist_feats), axis=1)
    return data

In [10]:
target_idx = config.target_idx

In [11]:
dataset = QM9('../datasets/qm9_geometric_work/', transform=transform)

In [12]:
dataset = dataset.shuffle()
train_dataset = dataset[:config.n_train]
valid_dataset = dataset[config.n_train:config.n_train+config.n_valid]
test_dataset = dataset[config.n_train+config.n_valid:]
config.n_test = len(test_dataset)
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, pin_memory=True, num_workers=config.num_workers)
valid_loader = DataLoader(valid_dataset, batch_size=config.batch_size, pin_memory=True, num_workers=config.num_workers)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=config.num_workers)

In [13]:
import os
os.makedirs(config.model_dir, exist_ok=True)

In [14]:
torch.save(dataset.indices(), config.model_dir+'dataset_indices.pt')

In [15]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [16]:
def score(loader, model, device=device):
    model = model.to(device)
    pbar = tqdm.tqdm(enumerate(loader, 0))
    maes = []
    for i, data in pbar:
        data.to(device)
        prediction = model(data)
        mae = (prediction.view(-1) - data.y[:, target_idx]).abs()
        maes += [mae.detach().cpu()]
    maes = torch.cat(maes, dim=0)
    mae = maes.mean().item()
    return mae/chemical_accuracy[config.target_idx]

# Model

In [17]:
target_vec = []

In [18]:
# based on https://schnetpack.readthedocs.io/en/stable/tutorials/tutorial_02_qm9.html
# and https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/models/schnet.html#SchNet
for data in train_loader:
    data = data.to(device)
    atomU0s = torch.tensor(qm9.atomrefs[target_idx], device=device)[torch.argmax(data.x[:, :5], axis=1)]
    target_modular = scatter(atomU0s, data.batch, dim=-1, reduce='sum')
    target_vec += [(data.y[:, target_idx] - target_modular).detach().cpu().numpy()]
target_vec = np.concatenate(target_vec, axis=0)

In [19]:
target_mean = np.mean(target_vec)
target_std = np.std(target_vec)

In [20]:
class GATNN(nn.Module):
    def __init__(self, vert_dim, edge_dim, hidden_dim=config.hidden_dim, nhead=config.nhead, nlayers=config.nlayers,
                 #ro_dmodel=128, ro_dim_feedforward=512, ro_nlayers=6, ro_nhead=8, 
                 mean=None, std=None, atomref=None,
                 max_vertices=29, max_edges=28, use_pos=False):
        """
        transfomer model is some type of transformer that 
        """
        super(GATNN, self).__init__()
        self.hidden_dim = hidden_dim
        gat_layers = []
        for _ in range(nlayers):
            msg = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), 
                                      nn.ReLU(inplace=True),
                                      nn.Linear(hidden_dim, hidden_dim),
                                      nn.ReLU(inplace=True),
                                      nn.Linear(hidden_dim, hidden_dim))
            gat_layers += [(tgnn.TransformerConv(hidden_dim, hidden_dim), 'x, edge_index, edge_attr -> x')]
        
        self.gat_layers = tgnn.Sequential('x, edge_index, edge_attr', gat_layers)
        # gate turns the hidden states into attention weights
        self.gate = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), 
                                      nn.ReLU(inplace=True),
                                      nn.Linear(hidden_dim, hidden_dim),
                                      nn.ReLU(inplace=True),
                                      nn.Linear(hidden_dim, 1)) # sequential instead of linear also good improvement
        # fc_out turns the hidden states into predictions
        self.fc_out = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), 
                                      nn.ReLU(inplace=True),
                                      nn.Linear(hidden_dim, hidden_dim),
                                      nn.ReLU(inplace=True),
                                      nn.Linear(hidden_dim, 1)) # sequential instead of linear huge improvment
        self.readout = tgnn.GlobalAttention(self.gate, None)
        
        self.x_emb = nn.Linear(vert_dim, hidden_dim)
        if use_pos:
            self.pos_emb = nn.Linear(3, hidden_dim)
        else:
            self.pos_emb = None
        self.edge_emb = nn.Linear(edge_dim, hidden_dim)
        
        #TODO: maybe:
        #self.readout = SelfAttentionReadout(ro_dmodel, ro_dim_feedforward, ro_nlayers, ro_nhead)
        
        self.mean = mean
        self.std = std
        self.register_buffer('initial_atomref', atomref)
        self.atomref = None
        if atomref is not None:
            self.atomref = nn.Embedding(100, 1)
            self.atomref.weight.data.copy_(atomref)
    
    
    def forward(self, data):
        x, z, edge_index, batch = data.x, data.z, data.edge_index, data.batch
        h = self.x_emb(x) 
        if self.pos_emb is not None:
            h += self.pos_emb(data.pos)
        edge_attr = self.edge_emb(data.edge_attr).view(-1, 1, self.hidden_dim)
        h = self.gat_layers(h, edge_index, edge_attr) 
        out = self.readout(h, batch)
        out = self.fc_out(out)
        
        
        #batch = batch.permute(1, 0, 2) # seq_dim x batch_dim x n_model
        #transformer_out = self.enc(batch)
        #out = self.fc_out(transformer_out[0]) 
        
        # tricks from Schnet
        if self.mean is not None and self.std is not None:
            out = out * self.std + self.mean
                
        if self.atomref is not None:
            out = out + tgnn.global_add_pool(self.atomref(z), batch)
        
        return out

In [21]:
data = next(iter(train_loader))
model = GATNN(data.x.shape[1], data.edge_attr.shape[1], atomref=dataset.atomref(target_idx), 
              mean=target_mean, std=target_std, use_pos=config.use_pos)
loss = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=config.lr)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True, patience=config.patience, factor=config.factor)
early_stopping = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint.pt')

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


# Training

In [22]:
model = model.to(device)

In [24]:
loss_hist = []
try:
    # For each epoch
    for epoch in range(config.n_epochs):
        # For each batch in the dataloader
        pbar = tqdm.tqdm(enumerate(train_loader, 0))
        epoch_loss = 0
        for i, data in pbar:
            model.zero_grad()
            data.to(device)
            target = data.y[:, target_idx]
            prediction = model(data)
            output = loss(prediction.view(-1), target)
            mae = (prediction.view(-1) - target).abs().mean()
            epoch_loss = (epoch_loss*i + mae.item())/(i+1)
            
            pbar.set_description('Epoch %d: MAE/CA %2.6f'%(epoch+1, epoch_loss/chemical_accuracy[target_idx]))
            output.backward()
            optimizer.step()
            wandb.log({'MSE': output.item()})
        
        valid_loss = score(valid_loader, model)
        curr_lr = list(optimizer.param_groups)[0]['lr']
        wandb.log({'MAE':epoch_loss, 
                   'MAE/CA':epoch_loss/chemical_accuracy[target_idx],
                   'learning rate':curr_lr,
                   'MAE/CA valid':valid_loss})
        
        lr_scheduler.step(epoch_loss)
        early_stopping(valid_loss, model)
        loss_hist += [epoch_loss] 
        
        if early_stopping.early_stop:
            break
        
        if curr_lr < config.minimal_lr:
            break


except KeyboardInterrupt:
    print('keyboard interrupt caught')
    torch.save(model.state_dict(), config.model_dir+'gat_epoch%d.pt'%(epoch+1))

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 1: MAE/CA 81.865731: : 430it [00:36, 11.89it/s] 
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
40it [00:01, 30.07it/s]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

EarlyStopping counter: 1 out of 200



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 4: MAE/CA 66.688480: : 430it [00:26, 16.21it/s]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
0it [00:00, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
40it [00:01, 25.94it/s]

EarlyStopping counter: 2 out of 200



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 5: MAE/CA 65.633392: : 430it [00:28, 14.88it/s]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
0it [00:00, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
37it [00:01, 31.45it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
40it [00:01, 27.23it/s]
[W pthr

...

EarlyStopping counter: 8 out of 200



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 789: MAE/CA 2.388801: : 430it [00:22, 19.33it/s]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
40it [00:01, 21.08it/s]

EarlyStopping counter: 9 out of 200



[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch 790: MAE/CA 2.375985: : 305it [00:15, 19.24it/s]

keyboard interrupt caught


In [25]:
pbar = tqdm.tqdm(enumerate(test_loader, 0))
epoch_loss = 0
maes = []
for i, data in pbar:
    data.to(device)
    prediction = model(data)
    mae = (prediction.view(-1) - data.y[:, target_idx]).abs()
    maes += [mae.detach().cpu()]
maes = torch.cat(maes, dim=0)
mae = maes.mean().item()
print(mae, mae/chemical_accuracy[target_idx])
wandb.log({'TEST MAE':mae, 'TEST MAE/CA':mae/chemical_accuracy[target_idx]})


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
0it [00:00, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
339it [00:06, 50.60it/s]

0.12964433431625366 3.0149845189826436
